In [2]:
import os
# os.chdir('C:\\Users\\CrankMuffler\\Documents\\GitHub\\RA2_alpine_lads')
os.chdir('/mnt/jw01-aruk-home01/projects/ra_challenge/RA_challenge/RA2_alpine_lads')

In [4]:
from utils.config import Config

# test_config = Config('./utils/stadlerm_config.json')

test_config = Config()

In [5]:
from dataset.train_dataset import train_dataset

dataset = train_dataset(test_config)

hands_dataset, feet_dataset, hands_dataset_val, feet_dataset_val = dataset.initialize_pipeline()

In [6]:
feet_dataset

<PrefetchDataset shapes: ((None, 512, 378, 1), (None, 13)), types: (tf.float32, tf.int64)>

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt

In [8]:
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(test_config.img_height,test_config.img_width ,1)),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(13)
])

In [9]:
model.compile(loss='mean_squared_error', optimizer='adam')

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 512, 378, 16)      160       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 256, 189, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 256, 189, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 128, 94, 32)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 94, 64)       18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 64, 47, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 192512)            0

In [11]:
history = model.fit(
    feet_dataset,validation_data=feet_dataset_val,epochs=10, steps_per_epoch=50,
    validation_steps=2
)

Train for 50 steps, validate for 2 steps
Epoch 1/10
50/50 [==============================] - 25s 501ms/step - loss: 4.7240 - val_loss: 8.5109
Epoch 2/10
50/50 [==============================] - 9s 189ms/step - loss: 3.9792 - val_loss: 8.3942
Epoch 3/10
50/50 [==============================] - 9s 175ms/step - loss: 3.9807 - val_loss: 8.2334
Epoch 4/10
50/50 [==============================] - 9s 186ms/step - loss: 4.0716 - val_loss: 8.3896
Epoch 5/10
50/50 [==============================] - 9s 185ms/step - loss: 3.6644 - val_loss: 9.0917
Epoch 6/10
50/50 [==============================] - 9s 180ms/step - loss: 3.9685 - val_loss: 8.5898
Epoch 7/10
50/50 [==============================] - 9s 189ms/step - loss: 3.3936 - val_loss: 8.6102
Epoch 8/10
22/50 [============>.................] - ETA: 5s - loss: 3.8616

KeyboardInterrupt: 

In [19]:
for image, label in feet_dataset_val.take(5):
    N = image.shape[0]
    D = label.shape[1]
    
    error = np.zeros((N, D))
    
    for n in range(N):
        y_pred = model.predict(np.array(image[n]).reshape(1,test_config.img_height,test_config.img_width,1))
        y = label[n]
        
        
        E[n, :] = np.square(y_pred - y)
        
    rmse = np.sqrt(np.mean(E, axis = 0))
        
    print("Batch RMSE: ", rmse)

Batch RMSE:  [ 0.90138782  1.08972474  1.03077641  1.11803399  1.          1.03077641
  1.03077641  0.70710678  1.11803399  1.22474487  1.11803399  1.11803399
 10.43132302]
Batch RMSE:  [0.         0.         0.75       0.90138782 0.5        1.11803399
 1.         0.90138782 0.90138782 0.70710678 0.90138782 0.70710678
 6.03634823]
Batch RMSE:  [ 0.8291562   0.79056942  1.08972474  1.19895788  1.03077641  1.25
  1.3462912   1.          1.22474487  1.22474487  1.22474487  1.32287566
 11.14955156]
Batch RMSE:  [ 0.8291562   0.75        1.27475488  1.52069063  1.22474487  1.88745861
  1.14564392  1.36930639  1.62018517  1.41421356  1.62018517  1.41421356
 12.97593927]
Batch RMSE:  [ 0.79056942  0.79056942  1.11803399  1.11803399  1.11803399  1.41421356
  1.03077641  1.03077641  1.25        1.11803399  1.25        1.22474487
 11.21383075]
